Property Graph from Arrows App

```
CREATE (:Person {readUser: "", gender: "", age: "", ageGroup: ""})-[:INVOLVED_IN]->(n0:Crash {crashID: "", crashType: "", numberFatalities: "", busInvolvement: "", heavyRigidTruckInvolvement: "", articulatedTruckInvolvement: "", speedLimit: ""})-[:OCCURED_AT]->(:Location {state: "", nationalRemoteAreas: "", `sa4Name`: "", lgaName: "", nationalRoadType: ""}),
(n0)-[:HAPPENED_AT]->(:DateTime {month: "", year: "", dayOfWeek: "", time: "", timeOfDay: "", weekdayOrWeekend: "", christmasPeriod: "", easterPeriod: ""})
```